In [1]:
import sys
import importlib
from types import SimpleNamespace
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.special import softmax
from joblib import Parallel, delayed
import seaborn as sns

sys.path.append("../src")
sys.path.append("../configs")

sys.argv = ['--config', 'config1']

from models import *
from loss import *
from train import *
from data import *

def gpu_unravel(batch):
        input_dict, target_dict = batch
        input_dict = {k: input_dict[k].cuda() for k in input_dict}
        target_dict = {k: target_dict[k].cuda() for k in target_dict}
        return input_dict, target_dict
    
dict_unravel = gpu_unravel

Using config file config1


In [2]:
pwd

'/home/ubuntu/kaggle/kaggle-landmark-recognition-2020-1st-place/notebooks'

In [2]:
name = "config1"
# pretrained_weights = "../models/config1_ckpt_10.pth"
pretrained_weights = "../models/config1/config1_ckpt_10.pth"

csv = "train"

#このtrainはGLRv2のfull trainだと思う。
# train = pd.read_csv(f"../embeddings/{csv}.csv")
train = pd.read_csv(f"../../input/GLDv2/{csv}.csv")

# train["img_folder"] = "/ssd/kaggle-landmark/input/train/"
train["img_folder"] = "../../input/GLDv2/train/"
# train["img_folder"] = "/home/ubuntu/kaggle/input/GLDv2/train/"
train["target"] = 0

In [3]:
train

,id,url,landmark_id,img_folder,target
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820,../../input/GLDv2/train/,0
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169,../../input/GLDv2/train/,0
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914,../../input/GLDv2/train/,0
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140,../../input/GLDv2/train/,0
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474,../../input/GLDv2/train/,0
...,...,...,...,...,...
4132909,fc0f007893b11ba7,https://upload.wikimedia.org/wikipedia/commons...,172138,../../input/GLDv2/train/,0
4132910,39aad18585867916,https://upload.wikimedia.org/wikipedia/commons...,162860,../../input/GLDv2/train/,0
4132911,fd0725460e4ebbec,https://upload.wikimedia.org/wikipedia/commons...,191243,../../input/GLDv2/train/,0
4132912,73691ae29e24ba19,https://upload.wikimedia.org/wikipedia/commons...,145760,../../input/GLDv2/train/,0


In [4]:
aug = A.Compose([ 
                A.SmallestMaxSize(512),
                A.CenterCrop(always_apply=False, p=1.0, height=512, width=512), 
                ], 
                p=1.0
                )

In [6]:
64*3

192

In [5]:
val_ds = GLRDataset(train, normalization=args.normalization, aug=aug)

# batch_size = 256
batch_size = 16
val_dl = DataLoader(dataset=val_ds,
                    batch_size=batch_size,
                    sampler=SequentialSampler(val_ds), collate_fn=collate_fn, num_workers=40, pin_memory=True)

In [ ]:
img = cv2.imread("/home/ubuntu/kaggle/input/GLDv2/train/"

In [19]:
val_ds.load_one('312777185f4d71f1',"/home/ubuntu/kaggle/input/GLDv2/train/")

array([[[ 28,  67, 124],
        [ 30,  69, 126],
        [ 32,  71, 128],
        ...,
        [ 29,  65, 123],
        [ 27,  63, 121],
        [ 27,  63, 121]],

       [[ 31,  70, 127],
        [ 31,  70, 127],
        [ 32,  71, 128],
        ...,
        [ 27,  63, 121],
        [ 28,  64, 122],
        [ 29,  65, 123]],

       [[ 32,  71, 128],
        [ 32,  71, 128],
        [ 32,  71, 128],
        ...,
        [ 29,  65, 123],
        [ 29,  65, 123],
        [ 30,  66, 124]],

       ...,

       [[ 21,  25,  26],
        [ 23,  27,  28],
        [ 16,  17,  19],
        ...,
        [ 10,  11,  13],
        [ 11,  12,  14],
        [ 13,  14,  16]],

       [[ 24,  28,  31],
        [  9,  13,  14],
        [ 11,  12,  14],
        ...,
        [  7,   7,   9],
        [  7,   7,   9],
        [  8,   8,  10]],

       [[ 29,  33,  36],
        [ 14,  18,  21],
        [ 15,  16,  18],
        ...,
        [ 13,  13,  15],
        [ 15,  15,  17],
        [ 17,  17,  19]]

In [16]:
aug

Compose([
  SmallestMaxSize(always_apply=False, p=1, max_size=512, interpolation=1),
  CenterCrop(always_apply=False, p=1.0, height=512, width=512),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})

In [15]:
val_ds

In [6]:
model = Net(args)
model.eval()
model.cuda()

model.load_state_dict(torch.load(pretrained_weights))

model = nn.DataParallel(model)


In [7]:
pwd

'/home/ubuntu/kaggle/kaggle-landmark-recognition-2020-1st-place/notebooks'

In [8]:
def get_embeddings(dl, model):
    with torch.no_grad():
        embeddings = np.zeros((len(dl.dataset) , 512))
        total = len(dl)
        for idx, batch in tqdm(enumerate(dl), total=len(dl)):
            input_dict, target_dict = dict_unravel(batch)

            outs = model.forward(input_dict, get_embeddings=True)["embeddings"]

            embeddings[idx*batch_size:idx*batch_size+outs.size(0),:] = outs.detach().cpu().numpy()

    return embeddings
#landmark_id to calss_idは不要か。
embeddings = get_embeddings(val_dl, model)
print(embeddings.shape)
np.save(f"../embeddings/{name}_{csv}_embeddings", embeddings)

  0%|          | 0/16145 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
embeddings.shape
np.save(f"../embeddings/{name}_{csv}_embeddings", embeddings)